In [ ]:
import sys
import json
import pandas as pd
from datasets import load_metric

In [ ]:
# dts = "training"
dts = "validation"
path = 'MuP_dataset/'+dts+'_complete.jsonl'
with open(path, 'r') as json_file:
    json_list = list(json_file)

In [ ]:
print(type(json_list))
print(len(json_list))

In [ ]:
i = 0
for json_str in json_list:
    result = json.loads(json_str)
    # print(f"result: {result}")
    i += 1
    n=30
    if i<n and i >= n-5:
        print(isinstance(result, dict))
        # print(len(result))
        # print(result.keys())
        print(result["summary"])
    if i>n:
        break

In [ ]:
col_name = ["paper_id","summary"]
summary_df = pd.DataFrame(columns=col_name)
for json_str in json_list:
    result = json.loads(json_str)
    df = pd.DataFrame([[result["paper_id"], result["summary"]]], columns=col_name)
    summary_df = pd.concat([summary_df,df])
summary_df

In [ ]:
num_paper = summary_df.groupby(['paper_id']).count()
num_paper['num_paper'] = 1
num_paper.groupby(['summary']).count()

In [ ]:
num_paper.drop('num_paper', inplace=True,axis=1)
num_paper.sort_values(["summary"])

In [ ]:
list_paper_id = []
for i in range(1, max(num_paper['summary'])+1):
    list_paper_id.append(list((num_paper[num_paper["summary"]==i]).index))
for id in list_paper_id:
    print(len(id))

In [ ]:
df_list = []
for i in range(len(list_paper_id)):
    df_list.append(summary_df[summary_df.paper_id.isin(list_paper_id[i])].sort_values("paper_id"))
df_list[1][:20]

## 2 Summaries

In [ ]:
rouge_score = load_metric("rouge")

In [ ]:
# df_sum2 = df_list[1]
# score_col = ["paper_id", "R1-F1", "R2-F1", "RL-F1", "RLsum-F1"]
# df_score2 = pd.DataFrame(columns=score_col)
# for id in list_paper_id[1]:
#     sumaries = list(df_list[1][df_list[1]["paper_id"]==id]["summary"])
#     score = rouge_score.compute(
#         predictions=[sumaries[0]], references=[sumaries[1]], use_stemmer=False
#     )
#     df_score2 = pd.concat([df_score2, pd.DataFrame(
#         [[
#             id,
#             score['rouge1'][0][2],
#             score['rouge2'][0][2],
#             score['rougeL'][0][2],
#             score['rougeLsum'][0][2]
#         ]], 
#         columns=score_col)])
# df_score2

In [ ]:
# df_score2.min()

In [ ]:
# df_score2.to_csv("visualization_data/"+dts+"_rouge_2sum.csv")

## Multi Summaries

In [ ]:
def n_scores(n):
    pairs = []
    for i in range(n):
        for j in range (i+1,n):
            pairs.append(str(i+1)+"-"+str(j+1))
    col = pd.MultiIndex.from_product([["R1-F1", "R2-F1", "RL-F1", "RLsum-F1"], pairs])
    scores = pd.DataFrame(columns=col)
    scores.insert(0, "paper_id", list_paper_id[n-1])
    scores.set_index("paper_id", inplace=True)
    return scores
# n_scores(3).head()

In [ ]:
def rouge_cal(n):
    df_scores = n_scores(n)
    for num_id,id in enumerate(list_paper_id[n-1]):
        sumaries = list(df_list[n-1][df_list[n-1]["paper_id"]==id]["summary"])
        sys.stdout.write("\r" + str(num_id) + "/" + str(len(list_paper_id[n-1])))
        sys.stdout.flush()
        # print(id,"/",len(list_paper_id[n-1]), flush=True)
        for i in range(n):
            for j in range(i+1,n):
                score = rouge_score.compute(
                    predictions=[sumaries[i]], references=[sumaries[j]], use_stemmer=False
                )
                pair = str(i+1)+"-"+str(j+1)
                df_scores.loc[id]['R1'][pair] = score['rouge1'][0][2]
                df_scores.loc[id]['R2'][pair] = score['rouge2'][0][2]
                df_scores.loc[id]['RL'][pair] = score['rougeL'][0][2]
                df_scores.loc[id]['RLsum'][pair] = score['rougeLsum'][0][2]
                # break
            # break
        # break
    # df_score2
    # len(list_paper_id)
    return df_scores

In [ ]:
for n in range(3,len(list_paper_id)):
    n = 4
    print("Calculating ROUGE on", n, "summaries")
    df_scores = rouge_cal(n)
    df_scores.to_csv("visualization_data/"+dts+"_rouge_"+str(n)+"sum.csv")
    break